<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Comparison-plots" data-toc-modified-id="Comparison-plots-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Comparison plots</a></span></li></ul></div>

In [ ]:
from math import isqrt
import os.path
from time import perf_counter

import numpy as np
from scipy.special import gamma, gammaln, gammainc
import scipy.linalg as la
import scipy.integrate as spi
import matplotlib
import matplotlib.pyplot as plt
import netCDF4 as nc4
import matplotlib.animation as animation

In [ ]:
kc_long = 9.44e9 # cm^3/g^2/s
kr_long = 5.78e3 # cm^3/g/s
kc_si = kc_long # m^3/kg^2/s is the same
kr_si = kr_long * 1.e-3 # m^3/kg/s
rho_air = 1.2 # kg/m^3
rho_water = 1000. # kg/m^3
mass_100_micron_diameter = rho_water * (np.pi/6.) * 1.e-4**3 # kg
kc = kc_si * rho_air * mass_100_micron_diameter**2 # kg/s
kr = kr_si * rho_air * mass_100_micron_diameter # kg/s
kr_over_kc = kr/kc

In [ ]:
def diameter_to_scaled_mass(d):
    return (d*1.e4)**3
def scaled_mass_to_diameter(x):
    return 1.e-4 * x**(1./3.)

In [ ]:
def add_logs(x, y):
    """Returns log(exp(x)+exp(y))."""
    return x + np.log(1. + np.exp(y-x))

def sub_logs(x, y):
    """Returns log(exp(x)-exp(y))."""
    assert y < x
    return x + np.log(1. - np.exp(y-x))

def height(lx, ly1, ly2, lz1, lz2):
    """Range of possible ly values such that add_logs(lx, ly) is in range for lz."""
    bottom = ly1
    if lz1 > lx:
        bottom = max(bottom, sub_logs(lz1, lx))
    return max(0., min(ly2, sub_logs(lz2, lx)) - bottom)

def calc_area(vertex_lxs, heights):
    dlxs = vertex_lxs[1:] - vertex_lxs[:-1]
    avg_hts = 0.5 * (heights[1:] + heights[:-1])
    return np.dot(dlxs, avg_hts)

def long_kernel_top_hat(kr_over_kc, lx1, lx2, ly1, ly2, lz1, lz2):
    """Integrated nondimensionalized Long's (1974) kernel over a pair of top hat bins.

    Arguments:
    kr_over_kc - kr/kc
    lx1, lx2 - Logarithm of bounds of bin of first particle.
    ly1, ly2 - Logarithm of bounds of bin of second particle.
    lz1, lz2 - Logarithm of bounds of bin of outgoing particle.
    Output:
    Long's kernel in a nondimensionalized representation.

    Nondimensionalized representation in this case means scaling masses such that the
    the mass of a particle on the rain-cloud boundary (100 micron diameter) is 1, and
    kc = 1.

    For efficiency in the mass-conserving scheme, we use an asymmetric kernel; the
    result represents the mass moved from the first bin to the outgoing bin,
    rather than the number of collisions.
    """
    mean_lx = (lx1 + lx2) / 2
    mean_ly = (ly1 + ly2) / 2
    # Area of intersection calculation
    min_output = add_logs(lx1, ly1)
    max_output = add_logs(lx2, ly2)
    if min_output > lz2 or max_output < lz1:
        return 0.
    if lz1 > ly2:
        start = max(lx1, sub_logs(lz1, ly2))
    else:
        start = lx1
    end = min(lx2, sub_logs(lz2, ly1))
    vertex_lxs = [start, end]
    if lz1 > ly1:
        low_intersect = sub_logs(lz1, ly1)
        if start < low_intersect < end:
            vertex_lxs.append(low_intersect)
    if lz2 > ly2:
        high_intersect = sub_logs(lz2, ly2)
        if start < high_intersect < end:
            vertex_lxs.append(high_intersect)
    vertex_lxs = np.array(sorted(vertex_lxs))
    heights = np.array([height(lx, ly1, ly2, lz1, lz2) for lx in vertex_lxs])
    area = calc_area(vertex_lxs, heights)
    if mean_lx < 0. and mean_ly < 0.:
        return area * (np.exp(2.*mean_lx - mean_ly) + np.exp(mean_ly))
    else:
        return area * kr_over_kc * (np.exp(mean_lx - mean_ly) + 1)

In [ ]:
def find_bin(bin_bounds, val, check_bottom=True):
    if check_bottom:
        assert val > bin_bounds[0]
    else:
        if val <= bin_bounds[0]:
            return -1
    for i in range(1, len(bin_bounds)):
        if val < bin_bounds[i]:
            return i-1
    return -1

In [ ]:
d_min = 1.e-6
d_max = 1.e-3
x_min = diameter_to_scaled_mass(d_min)
x_max = diameter_to_scaled_mass(d_max)
lx_min = np.log(x_min)
lx_max = np.log(x_max)
num_bins = 90
bin_bounds = np.linspace(lx_min, lx_max, num_bins+1)
dls = bin_bounds[1:] - bin_bounds[:-1]
sum_idxs = np.zeros((num_bins, num_bins), dtype=np.int32)
num_sum_bins = np.zeros((num_bins, num_bins), dtype=np.int32)
for i in range(num_bins):
    for j in range(num_bins):
        low_sum_log = add_logs(bin_bounds[i], bin_bounds[j])
        high_sum_log = add_logs(bin_bounds[i+1], bin_bounds[j+1])
        low_sum_idx = find_bin(bin_bounds, low_sum_log)
        if low_sum_idx == -1:
            low_sum_idx = num_bins-1
        high_sum_idx = find_bin(bin_bounds, high_sum_log)
        if high_sum_idx == -1:
            high_sum_idx = num_bins-1
        sum_idxs[i,j] = low_sum_idx
        num_sum_bins[i,j] = high_sum_idx - low_sum_idx + 1
max_sum_bins = num_sum_bins.max()
kernel = np.zeros((num_bins, num_bins, max_sum_bins))
# Using num_bins-1 here causes the top bin to be non-interacting with the others.
for i in range(num_bins):
    for j in range(num_bins):
        idx = sum_idxs[i,j]
        for k in range(num_sum_bins[i,j]):
            if idx+k == num_bins-1:
                high_bound = np.inf
            else:
                high_bound = bin_bounds[idx+k+1]
            kernel[i,j,k] = long_kernel_top_hat(kr_over_kc, bin_bounds[i], bin_bounds[i+1],
                                                bin_bounds[j], bin_bounds[j+1],
                                                bin_bounds[idx+k], high_bound)

In [ ]:
mass_init = 2.e-3 # kg/kg mass mixing ratio
nc_init = 400. # cm^-3 number concentration
nu_init = 5. # shape parameter
m3_scaled_init = mass_init / mass_100_micron_diameter # kg^-1
m3_init = mass_init / (rho_water * np.pi/6.) # m^3 / kg 3rd moment
m0_init = nc_init * 1.e6 * rho_air # kg^-1 number concentration
lambda_init = ( m0_init * gamma(nu_init + 3)/ (m3_init * gamma(nu_init)) )**(1./3.) # m^-1 scale parameter

In [ ]:
bin_bounds_d = np.array([scaled_mass_to_diameter(np.exp(lx)) for lx in bin_bounds])
def f_initialize_gamma(bin_bounds_d, dls, lam, nu):
    gamma_integrals = gammainc(nu+3., lam*bin_bounds_d)
    return (gamma_integrals[1:] - gamma_integrals[:-1]) / dls

def dfdlambda_initialize(bin_bounds_d, dls, lam, nu):
    bin_func = bin_bounds_d * (lam*bin_bounds_d)**(nu+2) * np.exp(-lam*bin_bounds_d) / gamma(nu+3)
    return (bin_func[1:] - bin_func[:-1]) / dls

def dfdnu_initialize(bin_bounds_d, dls, lam, nu):
    # Use finite difference with Richardson extrapolation.
    perturb = 1.e-6
    return (4.*f_initialize_gamma(bin_bounds_d, dls, lam, nu+perturb) -
            f_initialize_gamma(bin_bounds_d, dls, lam, nu+2.*perturb) -
            3.*f_initialize_gamma(bin_bounds_d, dls, lam, nu)           ) / (2.*perturb)

In [ ]:
f_init = f_initialize_gamma(bin_bounds_d, dls, lambda_init, nu_init)
bin_midpoints = 0.5 * (bin_bounds[:-1] + bin_bounds[1:])
plot_lxs = 2. + bin_midpoints / (3.*np.log(10))
fig = plt.figure(figsize=(5, 4))
plt.plot(plot_lxs, f_init)

In [ ]:
dfdlambda_init = dfdlambda_initialize(bin_bounds_d, dls, lambda_init, nu_init)
perturb = 1.e-5 * lambda_init
f_perturb = f_initialize_gamma(bin_bounds_d, dls, lambda_init + perturb, nu_init)
dfdlambda_approx = (f_perturb-f_init)/perturb
fig = plt.figure(figsize=(5, 4))
plt.plot(plot_lxs, (dfdlambda_init - dfdlambda_approx)/dfdlambda_init.max())

In [ ]:
dfdnu_init = dfdnu_initialize(bin_bounds_d, dls, lambda_init, nu_init)
perturb = 1.e-5 * nu_init
f_perturb = f_initialize_gamma(bin_bounds_d, dls, lambda_init, nu_init + perturb)
dfdnu_approx = (f_perturb-f_init)/perturb
fig = plt.figure(figsize=(5, 4))
plt.plot(plot_lxs, (dfdnu_init - dfdnu_approx)/dfdnu_approx.max())

In [ ]:
y_init = np.zeros((3*num_bins,))
y_init[:num_bins] = f_init
y_init[num_bins:2*num_bins] = dfdlambda_init
y_init[2*num_bins:] = dfdnu_init

In [ ]:
def dfdt(t, f, dls, kernel, sum_idxs, num_sum_bins):
    num_bins = len(f)
    if len(f.shape) == 1:
        f = f[:,None]
    f_outer = np.dot(f, np.transpose(f))
    output = np.zeros((num_bins,))
    for i in range(num_bins):
        for j in range(num_bins):
            idx = sum_idxs[i,j]
            fprod = f_outer[i,j]
            for k in range(num_sum_bins[i,j]):
                dfdt_term = fprod * kernel[i,j,k]
                output[i] -= dfdt_term
                output[idx+k] += dfdt_term
    output /= dls
    return output

def dfdt_and_djdt(t, y, dls, kernel, sum_idxs, num_sum_bins):
    num_bins = len(y) // 3
    f = y[:num_bins,None]
    dfdlam = y[num_bins:2*num_bins,None]
    dfdnu = y[2*num_bins:,None]
    f_outer = np.dot(f, np.transpose(f))
    doutdlam = np.dot(f, np.transpose(dfdlam))
    doutdlam += np.transpose(doutdlam)
    doutdnu = np.dot(f, np.transpose(dfdnu))
    doutdnu += np.transpose(doutdnu)
    output = np.zeros((3*num_bins,))
    for i in range(num_bins):
        for j in range(num_bins):
            idx = sum_idxs[i,j]
            fprod = f_outer[i,j]
            dpdlam = doutdlam[i,j]
            dpdnu = doutdnu[i,j]
            for k in range(num_sum_bins[i,j]):
                dfdt_term = fprod * kernel[i,j,k]
                output[i] -= dfdt_term
                output[idx+k] += dfdt_term
                dtermdlam = dpdlam * kernel[i,j,k]
                output[num_bins+i] -= dtermdlam
                output[num_bins+idx+k] += dtermdlam
                dtermdnu = dpdnu * kernel[i,j,k]
                output[2*num_bins+i] -= dtermdnu
                output[2*num_bins+idx+k] += dtermdnu
    output[:num_bins] /= dls
    output[num_bins:2*num_bins] /= dls
    output[2*num_bins:] /= dls
    return output

In [ ]:
dfdt_init = dfdt(0., f_init, dls, kernel, sum_idxs, num_sum_bins)
fig = plt.figure(figsize=(5, 4))
plt.plot(plot_lxs, dfdt_init)

In [ ]:
end_time = 900.
num_time_steps = 60
dt = end_time / num_time_steps
end_time_scaled = end_time * m3_scaled_init * kc
t_eval = np.linspace(0., end_time_scaled, num_time_steps+1)
ivp_out = spi.solve_ivp(dfdt_and_djdt, (0., end_time_scaled),
                        y_init, args=(dls, kernel, sum_idxs, num_sum_bins),
                        t_eval=t_eval)

In [ ]:
mass_convert = mass_init * 1.e3 * (3.*np.log(10))
ymin = 0.
ymax = 0.4*mass_convert
fig = plt.figure(figsize=(5, 4))
ax = fig.add_subplot(autoscale_on=False,
                     xlim=(2. + lx_min/(3.*np.log(10)), 2. + lx_max/(3.*np.log(10))),
                     ylim=(ymin, ymax))
ax.set_xlabel("$log_{10}(D)$ ($D$ in microns)")
ax.set_ylabel("$dm/dlog_{10}(D)$ (g/kg)")
ax.grid()

line, = ax.plot(plot_lxs, f_init, 'o-', lw=2)
time_template = 'time = %.1fs'
time_text = ax.text(0.05, 0.9, '', transform=ax.transAxes)
plt.vlines(2., ymin, ymax, 'k')

def animate(i):
    thisy = mass_convert*ivp_out.y[:num_bins,i]
    line.set_data(plot_lxs, thisy)
    time_text.set_text(time_template % (i*dt))
    return line, time_text

ani = animation.FuncAnimation(
    fig, animate, len(t_eval), interval=dt*1000*(5./end_time), blit=True)
ani.save("mass_evolution.gif")
plt.show()

In [ ]:
ymin = -1.e-5
ymax = 1.e-5
fig = plt.figure(figsize=(5, 4))
ax = fig.add_subplot(autoscale_on=False,
                     xlim=(2. + lx_min/(3.*np.log(10)), 2. + lx_max/(3.*np.log(10))),
                     ylim=(ymin, ymax))
ax.set_xlabel("$log_{10}(D)$ ($D$ in microns)")
ax.grid()

line, = ax.plot(plot_lxs, f_init, 'o-', lw=2)
time_template = 'time = %.1fs'
time_text = ax.text(0.05, 0.9, '', transform=ax.transAxes)
plt.vlines(2., ymin, ymax, 'k')

def animate(i):
    thisy = ivp_out.y[num_bins:2*num_bins,i]
    line.set_data(plot_lxs, thisy)
    time_text.set_text(time_template % (i*dt))
    return line, time_text

ani = animation.FuncAnimation(
    fig, animate, len(t_eval), interval=dt*1000*(5./end_time), blit=True)
plt.show()

In [ ]:
ymin = -1.e-1
ymax = 1.e-1
fig = plt.figure(figsize=(5, 4))
ax = fig.add_subplot(autoscale_on=False,
                     xlim=(2. + lx_min/(3.*np.log(10)), 2. + lx_max/(3.*np.log(10))),
                     ylim=(ymin, ymax))
ax.set_xlabel("$log_{10}(D)$ ($D$ in microns)")
ax.grid()

line, = ax.plot(plot_lxs, f_init, 'o-', lw=2)
time_template = 'time = %.1fs'
time_text = ax.text(0.05, 0.9, '', transform=ax.transAxes)
plt.vlines(2., ymin, ymax, 'k')

def animate(i):
    thisy = ivp_out.y[2*num_bins:,i]
    line.set_data(plot_lxs, thisy)
    time_text.set_text(time_template % (i*dt))
    return line, time_text

ani = animation.FuncAnimation(
    fig, animate, len(t_eval), interval=dt*1000*(5./end_time), blit=True)
plt.show()

In [ ]:
perturb = 1.e-4 * lambda_init
f_perturb = f_initialize_gamma(bin_bounds_d, dls, lambda_init+perturb, nu_init)
end_time_perturb = dt * m3_scaled_init * kc
ivp_perturb = spi.solve_ivp(dfdt, (0., end_time_perturb),
                            f_perturb, args=(dls, kernel, sum_idxs, num_sum_bins),
                            t_eval=np.array([0., end_time_perturb]))
dfdlam_approx = (ivp_perturb.y[:,1] - ivp_out.y[:num_bins,1]) / perturb
fig = plt.figure(figsize=(5, 4))
plt.plot(plot_lxs, ivp_out.y[num_bins:2*num_bins,1])
plt.plot(plot_lxs, dfdlam_approx)

In [ ]:
perturb = 1.e-4 * nu_init
f_perturb = f_initialize_gamma(bin_bounds_d, dls, lambda_init, nu_init+perturb)
end_time_perturb = dt * m3_scaled_init * kc
ivp_perturb = spi.solve_ivp(dfdt, (0., end_time_perturb),
                            f_perturb, args=(dls, kernel, sum_idxs, num_sum_bins),
                            t_eval=np.array([0., end_time_perturb]))
dfdnu_approx = (ivp_perturb.y[:,1] - ivp_out.y[:num_bins,1]) / perturb
fig = plt.figure(figsize=(5, 4))
plt.plot(plot_lxs, ivp_out.y[2*num_bins:,1])
plt.plot(plot_lxs, dfdnu_approx)

In [ ]:
def calc_moment(f, n, m3, bin_bounds, dls, bin_midpoints, min_d, max_d):
    min_idx = find_bin(bin_midpoints, np.log(diameter_to_scaled_mass(min_d)), check_bottom=False) + 1
    max_idx = find_bin(bin_bounds, np.log(diameter_to_scaled_mass(max_d)), check_bottom=False)
    if max_idx == -1:
        max_idx = num_bins-1
    bin_midpoints_d = np.array([scaled_mass_to_diameter(np.exp(p)) for p in bin_midpoints[min_idx:max_idx+1]])
    mom_func = bin_midpoints_d**(n-3)
    int_func = m3 * f[min_idx:max_idx+1] * dls[min_idx:max_idx+1]
    return np.dot(int_func, mom_func)

In [ ]:
m3_init

In [ ]:
calc_moment(f_init, 3, m3_init, bin_bounds, dls, bin_midpoints, 1.e-6, 1.e-3)

In [ ]:
calc_moment(ivp_out.y[:num_bins,-1], 3, m3_init, bin_bounds, dls, bin_midpoints, 1.e-6, 1.e-3)

In [ ]:
def calc_moment_and_gradient(y, dfdt_val, n, m3, bin_bounds, dls, bin_midpoints, min_d, max_d):
    num_bins = len(dls)
    f = y[:num_bins]
    dfdlam = y[num_bins:2*num_bins]
    dfdnu = y[2*num_bins:]
    min_idx = find_bin(bin_midpoints, np.log(diameter_to_scaled_mass(min_d)), check_bottom=False) + 1
    max_idx = find_bin(bin_midpoints, np.log(diameter_to_scaled_mass(max_d)), check_bottom=False)
    if max_idx == -1:
        max_idx = num_bins-1
    bin_midpoints_d = np.array([scaled_mass_to_diameter(np.exp(p)) for p in bin_midpoints[min_idx:max_idx+1]])
    int_func = m3 * bin_midpoints_d**(n-3) * dls[min_idx:max_idx+1]
    mom = np.dot(f[min_idx:max_idx+1], int_func)
    grad = np.zeros((3,))
    grad[0] = np.dot(dfdt_val[min_idx:max_idx+1], int_func)
    grad[1] = np.dot(dfdlam[min_idx:max_idx+1], int_func)
    grad[2] = np.dot(dfdnu[min_idx:max_idx+1], int_func)
    return mom, grad / mom

In [ ]:
def autoconversion_accretion_and_grads(y, dfdt_val, dls, kernel, sum_idxs, num_sum_bins,
                                       bin_midpoints, cutoff_diameter, m3, tscale):
    max_cld_idx = find_bin(bin_midpoints, np.log(diameter_to_scaled_mass(cutoff_diameter)))
    assert max_cld_idx != -1
    num_bins = len(bin_midpoints)
    is_cloud = np.zeros((num_bins,), dtype=np.int8)
    is_cloud[:max_cld_idx+1] = True
    f = y[:num_bins,None]
    dfdlam = y[num_bins:2*num_bins,None]
    dfdnu = y[2*num_bins:,None]
    if len(dfdt_val.shape) == 1:
        dfdt_val = dfdt_val[:,None]
    f_outer = np.dot(f, np.transpose(f))
    doutdt = np.dot(f, np.transpose(dfdt_val))
    doutdt += np.transpose(doutdt)
    doutdlam = np.dot(f, np.transpose(dfdlam))
    doutdlam += np.transpose(doutdlam)
    doutdnu = np.dot(f, np.transpose(dfdnu))
    doutdnu += np.transpose(doutdnu)
    auto = 0.
    accr = 0.
    grad_auto = np.zeros((3,))
    grad_accr = np.zeros((3,))
    for i in range(num_bins):
        if not is_cloud[i]:
            break
        for j in range(num_bins):
            idx = sum_idxs[i,j]
            fprod = f_outer[i,j]
            dpdt = doutdt[i,j]
            dpdlam = doutdlam[i,j]
            dpdnu = doutdnu[i,j]
            for k in range(num_sum_bins[i,j]):
                if is_cloud[idx+k]:
                    continue
                dfdt_term = fprod * kernel[i,j,k]
                dtermdt = dpdt * kernel[i,j,k]
                dtermdlam = dpdlam * kernel[i,j,k]
                dtermdnu = dpdnu * kernel[i,j,k]
                if is_cloud[j]:
                    auto += dfdt_term
                    grad_auto[0] += dtermdt
                    grad_auto[1] += dtermdlam
                    grad_auto[2] += dtermdnu
                else:
                    accr += dfdt_term
                    grad_accr[0] += dtermdt
                    grad_accr[1] += dtermdlam
                    grad_accr[2] += dtermdnu
    auto *= m3 / tscale
    accr *= m3 / tscale
    grad_auto *= m3 / tscale
    grad_accr *= m3 / tscale
    return auto, accr, auto+accr, grad_auto / auto, grad_accr / accr, grad_auto+grad_accr / (auto+accr)

In [ ]:
tscale = 1. / (m3_scaled_init * kc)
mom0s = np.zeros((num_time_steps+1,))
mom3s = np.zeros((num_time_steps+1,))
mom6s = np.zeros((num_time_steps+1,))
mom9s = np.zeros((num_time_steps+1,))
mom_jacobian = np.zeros((num_time_steps+1, 4, 3))
autos = np.zeros((num_time_steps+1,))
accrs = np.zeros((num_time_steps+1,))
rps = np.zeros((num_time_steps+1,))
process_jacobian = np.zeros((num_time_steps+1, 3, 3))
for i in range(num_time_steps+1):
    y = ivp_out.y[:,i]
    dfdt_val = dfdt(0., y[:num_bins], dls, kernel, sum_idxs, num_sum_bins)
    mom0s[i], mom_jacobian[i,0,:] = calc_moment_and_gradient(y, dfdt_val, 0, m3_init, bin_bounds, dls,
                                                             bin_midpoints, 1.e-6, 1.e-3)
    mom3s[i], mom_jacobian[i,1,:] = calc_moment_and_gradient(y, dfdt_val, 3, m3_init, bin_bounds, dls,
                                                             bin_midpoints, 1.e-6, 1.e-3)
    mom6s[i], mom_jacobian[i,2,:] = calc_moment_and_gradient(y, dfdt_val, 6, m3_init, bin_bounds, dls,
                                                             bin_midpoints, 1.e-6, 1.e-3)
    mom9s[i], mom_jacobian[i,3,:] = calc_moment_and_gradient(y, dfdt_val, 9, m3_init, bin_bounds, dls,
                                                             bin_midpoints, 1.e-6, 1.e-3)
    proc_out = autoconversion_accretion_and_grads(y, dfdt_val, dls, kernel, sum_idxs, num_sum_bins,
                                                  bin_midpoints, 1.e-4, m3_init, tscale)
    autos[i], accrs[i], rps[i], process_jacobian[i,0,:], process_jacobian[i,1,:], process_jacobian[i,2,:] = proc_out

In [ ]:
fig = plt.figure(figsize=(5, 4))
plt.plot(t_eval * tscale, mom0s)

In [ ]:
fig = plt.figure(figsize=(5, 4))
plt.plot(t_eval * tscale, mom3s)

In [ ]:
fig = plt.figure(figsize=(5, 4))
plt.plot(t_eval * tscale, mom6s)

In [ ]:
fig = plt.figure(figsize=(5, 4))
plt.plot(t_eval * tscale, mom9s)

In [ ]:
fig = plt.figure(figsize=(5, 4))
plt.plot(t_eval * tscale, autos)

In [ ]:
fig = plt.figure(figsize=(5, 4))
plt.plot(t_eval * tscale, accrs)

In [ ]:
fig = plt.figure(figsize=(5, 4))
plt.plot(t_eval * tscale, rps)

In [ ]:
sigma_auto = np.zeros((num_time_steps+1,))
sigma_accr = np.zeros((num_time_steps+1,))
sigma_rp = np.zeros((num_time_steps+1,))
for i in range(num_time_steps+1):
    mom_inv = la.pinv(mom_jacobian[i,:,:])
    pj = process_jacobian[i,:,:]
    cov = pj @ mom_inv @ np.eye(4) @ mom_inv.T @ pj.T
    sigma_auto[i] = np.sqrt(cov[0,0])
    sigma_accr[i] = np.sqrt(cov[1,1])
    sigma_rp[i] = np.sqrt(cov[2,2])

In [ ]:
fig = plt.figure(figsize=(5, 4))
plt.plot(t_eval * tscale, sigma_auto)
plt.plot(t_eval * tscale, sigma_accr)
plt.plot(t_eval * tscale, sigma_rp)
for i in range(num_time_steps+1):
    if autos[i] < accrs[i]:
        time = t_eval[i] * tscale
        break
plt.vlines(time, 0., 30., 'k')
for i in range(num_time_steps+1):
    if autos[i] > 0.01 * autos.max():
        time = t_eval[i] * tscale
        break
plt.vlines(time, 0., 30., 'r', linestyle='--')
plt.ylim([0., 30.])

In [ ]:
momc0s = np.zeros((num_time_steps+1,))
momc3s = np.zeros((num_time_steps+1,))
momr0s = np.zeros((num_time_steps+1,))
momr3s = np.zeros((num_time_steps+1,))
mom_2cat_jacobian = np.zeros((num_time_steps+1, 4, 3))
for i in range(num_time_steps+1):
    y = ivp_out.y[:,i]
    dfdt_val = dfdt(0., y[:num_bins], dls, kernel, sum_idxs, num_sum_bins)
    momc0s[i], mom_2cat_jacobian[i,0,:] = calc_moment_and_gradient(y, dfdt_val, 0, m3_init, bin_bounds, dls,
                                                                   bin_midpoints, 1.e-6, 1.e-4)
    momc3s[i], mom_2cat_jacobian[i,1,:] = calc_moment_and_gradient(y, dfdt_val, 3, m3_init, bin_bounds, dls,
                                                                   bin_midpoints, 1.e-6, 1.e-4)
    momr0s[i], mom_2cat_jacobian[i,2,:] = calc_moment_and_gradient(y, dfdt_val, 0, m3_init, bin_bounds, dls,
                                                                   bin_midpoints, 1.e-4, 1.e-3)
    momr3s[i], mom_2cat_jacobian[i,3,:] = calc_moment_and_gradient(y, dfdt_val, 3, m3_init, bin_bounds, dls,
                                                                   bin_midpoints, 1.e-4, 1.e-3)

In [ ]:
fig = plt.figure(figsize=(5, 4))
plt.plot(t_eval * tscale, momc0s)

In [ ]:
fig = plt.figure(figsize=(5, 4))
plt.plot(t_eval * tscale, momc3s)

In [ ]:
fig = plt.figure(figsize=(5, 4))
plt.plot(t_eval * tscale, momr0s)

In [ ]:
fig = plt.figure(figsize=(5, 4))
plt.plot(t_eval * tscale, momr3s)

In [ ]:
sigma_auto_2cat = np.zeros((num_time_steps+1,))
sigma_accr_2cat = np.zeros((num_time_steps+1,))
sigma_rp_2cat = np.zeros((num_time_steps+1,))
for i in range(num_time_steps+1):
    mom_inv = la.pinv(mom_2cat_jacobian[i,:,:])
    pj = process_jacobian[i,:,:]
    cov = pj @ mom_inv @ np.eye(4) @ mom_inv.T @ pj.T
    sigma_auto_2cat[i] = np.sqrt(cov[0,0])
    sigma_accr_2cat[i] = np.sqrt(cov[1,1])
    sigma_rp_2cat[i] = np.sqrt(cov[2,2])

In [ ]:
fig = plt.figure(figsize=(5, 4))
plt.plot(t_eval * tscale, sigma_auto_2cat)
plt.plot(t_eval * tscale, sigma_accr_2cat)
plt.plot(t_eval * tscale, sigma_rp_2cat)
for i in range(num_time_steps+1):
    if autos[i] < accrs[i]:
        time = t_eval[i] * tscale
        break
plt.vlines(time, 0., 30., 'k')
for i in range(num_time_steps+1):
    if autos[i] > 0.01 * autos.max():
        time = t_eval[i] * tscale
        break
plt.vlines(time, 0., 30., 'r', linestyle='--')
plt.ylim([0., 30.])

In [ ]:
mom0s = np.zeros((num_time_steps+1,))
mom3s = np.zeros((num_time_steps+1,))
mom4s = np.zeros((num_time_steps+1,))
mom5s = np.zeros((num_time_steps+1,))
mom_small_jacobian = np.zeros((num_time_steps+1, 4, 3))
for i in range(num_time_steps+1):
    y = ivp_out.y[:,i]
    dfdt_val = dfdt(0., y[:num_bins], dls, kernel, sum_idxs, num_sum_bins)
    mom0s[i], mom_small_jacobian[i,0,:] = calc_moment_and_gradient(y, dfdt_val, 0, m3_init, bin_bounds, dls,
                                                             bin_midpoints, 1.e-6, 1.e-3)
    mom3s[i], mom_small_jacobian[i,1,:] = calc_moment_and_gradient(y, dfdt_val, 3, m3_init, bin_bounds, dls,
                                                             bin_midpoints, 1.e-6, 1.e-3)
    mom4s[i], mom_small_jacobian[i,2,:] = calc_moment_and_gradient(y, dfdt_val, 4, m3_init, bin_bounds, dls,
                                                             bin_midpoints, 1.e-6, 1.e-3)
    mom5s[i], mom_small_jacobian[i,3,:] = calc_moment_and_gradient(y, dfdt_val, 5, m3_init, bin_bounds, dls,
                                                             bin_midpoints, 1.e-6, 1.e-3)

In [ ]:
fig = plt.figure(figsize=(5, 4))
plt.plot(t_eval * tscale, mom4s)

In [ ]:
fig = plt.figure(figsize=(5, 4))
plt.plot(t_eval * tscale, mom5s)

In [ ]:
sigma_auto_small = np.zeros((num_time_steps+1,))
sigma_accr_small = np.zeros((num_time_steps+1,))
sigma_rp_small = np.zeros((num_time_steps+1,))
for i in range(num_time_steps+1):
    mom_inv = la.pinv(mom_small_jacobian[i,:,:])
    pj = process_jacobian[i,:,:]
    cov = pj @ mom_inv @ np.eye(4) @ mom_inv.T @ pj.T
    sigma_auto_small[i] = np.sqrt(cov[0,0])
    sigma_accr_small[i] = np.sqrt(cov[1,1])
    sigma_rp_small[i] = np.sqrt(cov[2,2])

In [ ]:
fig = plt.figure(figsize=(5, 4))
plt.plot(t_eval * tscale, sigma_auto_small)
plt.plot(t_eval * tscale, sigma_accr_small)
plt.plot(t_eval * tscale, sigma_rp_small)
for i in range(num_time_steps+1):
    if autos[i] < accrs[i]:
        time = t_eval[i] * tscale
        break
plt.vlines(time, 0., 30., 'k')
for i in range(num_time_steps+1):
    if autos[i] > 0.01 * autos.max():
        time = t_eval[i] * tscale
        break
plt.vlines(time, 0., 30., 'r', linestyle='--')
plt.ylim([0., 30.])

In [ ]:
mom_6_jacobian = np.zeros((num_time_steps+1, 6, 3))
mom_6_jacobian[:,0,:] = mom_small_jacobian[:,0,:]
mom_6_jacobian[:,3:,:] = mom_small_jacobian[:,1:,:]
mom1s = np.zeros((num_time_steps+1,))
mom2s = np.zeros((num_time_steps+1,))
for i in range(num_time_steps+1):
    y = ivp_out.y[:,i]
    dfdt_val = dfdt(0., y[:num_bins], dls, kernel, sum_idxs, num_sum_bins)
    mom1s[i], mom_6_jacobian[i,1,:] = calc_moment_and_gradient(y, dfdt_val, 1, m3_init, bin_bounds, dls,
                                                             bin_midpoints, 1.e-6, 1.e-3)
    mom2s[i], mom_6_jacobian[i,2,:] = calc_moment_and_gradient(y, dfdt_val, 2, m3_init, bin_bounds, dls,
                                                             bin_midpoints, 1.e-6, 1.e-3)

In [ ]:
sigma_auto_6 = np.zeros((num_time_steps+1,))
sigma_accr_6 = np.zeros((num_time_steps+1,))
sigma_rp_6 = np.zeros((num_time_steps+1,))
for i in range(num_time_steps+1):
    mom_inv = la.pinv(mom_6_jacobian[i,:,:])
    pj = process_jacobian[i,:,:]
    cov = pj @ mom_inv @ np.eye(6) @ mom_inv.T @ pj.T
    sigma_auto_6[i] = np.sqrt(cov[0,0])
    sigma_accr_6[i] = np.sqrt(cov[1,1])
    sigma_rp_6[i] = np.sqrt(cov[2,2])

In [ ]:
fig = plt.figure(figsize=(5, 4))
plt.plot(t_eval * tscale, sigma_auto_6)
plt.plot(t_eval * tscale, sigma_accr_6)
plt.plot(t_eval * tscale, sigma_rp_6)
for i in range(num_time_steps+1):
    if autos[i] < accrs[i]:
        time = t_eval[i] * tscale
        break
plt.vlines(time, 0., 30., 'k')
for i in range(num_time_steps+1):
    if autos[i] > 0.01 * autos.max():
        time = t_eval[i] * tscale
        break
plt.vlines(time, 0., 30., 'r', linestyle='--')
plt.ylim([0., 30.])

In [ ]:
def moment_rate_and_grad(n, y, dfdt_val, dls, kernel, sum_idxs, num_sum_bins,
                         bin_midpoints, min_d, max_d, m3, tscale):
    num_bins = len(bin_midpoints)
    min_idx = find_bin(bin_midpoints, np.log(diameter_to_scaled_mass(min_d)), check_bottom=False) + 1
    max_idx = find_bin(bin_midpoints, np.log(diameter_to_scaled_mass(max_d)), check_bottom=False)
    if max_idx == -1:
        max_idx = num_bins-1
    bin_midpoints_d = np.array([scaled_mass_to_diameter(np.exp(p)) for p in bin_midpoints])
    d_pow = bin_midpoints_d**(n-3)
    f = y[:num_bins,None]
    dfdlam = y[num_bins:2*num_bins,None]
    dfdnu = y[2*num_bins:,None]
    if len(dfdt_val.shape) == 1:
        dfdt_val = dfdt_val[:,None]
    f_outer = np.dot(f, np.transpose(f))
    doutdt = np.dot(f, np.transpose(dfdt_val))
    doutdt += np.transpose(doutdt)
    doutdlam = np.dot(f, np.transpose(dfdlam))
    doutdlam += np.transpose(doutdlam)
    doutdnu = np.dot(f, np.transpose(dfdnu))
    doutdnu += np.transpose(doutdnu)
    rate = 0.
    grad = np.zeros((3,))
    for i in range(num_bins):
        if i > max_idx:
            break
        for j in range(num_bins):
            idx = sum_idxs[i,j]
            fprod = f_outer[i,j]
            dpdt = doutdt[i,j]
            dpdlam = doutdlam[i,j]
            dpdnu = doutdnu[i,j]
            for k in range(num_sum_bins[i,j]):
                dfdt_term = fprod * kernel[i,j,k]
                dtermdt = dpdt * kernel[i,j,k]
                dtermdlam = dpdlam * kernel[i,j,k]
                dtermdnu = dpdnu * kernel[i,j,k]
                if i >= min_idx:
                    rate -= d_pow[i] * dfdt_term
                    grad[0] -= d_pow[i] * dtermdt
                    grad[1] -= d_pow[i] * dtermdlam
                    grad[2] -= d_pow[i] * dtermdnu
                if min_idx <= idx+k <= max_idx:
                    rate += d_pow[idx+k] * dfdt_term
                    grad[0] += d_pow[idx+k] * dtermdt
                    grad[1] += d_pow[idx+k] * dtermdlam
                    grad[2] += d_pow[idx+k] * dtermdnu
    rate *= m3 / tscale
    grad *= m3 / tscale
    return rate, grad / rate

In [ ]:
mom0_rates = np.zeros((num_time_steps+1,))
mom3_rates = np.zeros((num_time_steps+1,))
mom6_rates = np.zeros((num_time_steps+1,))
mom9_rates = np.zeros((num_time_steps+1,))
process_mom_jacobian = np.zeros((num_time_steps+1, 4, 3))
for i in range(num_time_steps+1):
    y = ivp_out.y[:,i]
    dfdt_val = dfdt(0., y[:num_bins], dls, kernel, sum_idxs, num_sum_bins)
    rate_out = moment_rate_and_grad(0, y, dfdt_val, dls, kernel, sum_idxs, num_sum_bins,
                                    bin_midpoints, 1.e-6, 1.e-3, m3_init, tscale)
    mom0_rates[i], process_mom_jacobian[i,0,:] = rate_out
    rate_out = moment_rate_and_grad(3, y, dfdt_val, dls, kernel, sum_idxs, num_sum_bins,
                                    bin_midpoints, 1.e-6, 1.e-3, m3_init, tscale)
    mom3_rates[i], process_mom_jacobian[i,1,:] = rate_out
    rate_out = moment_rate_and_grad(6, y, dfdt_val, dls, kernel, sum_idxs, num_sum_bins,
                                    bin_midpoints, 1.e-6, 1.e-3, m3_init, tscale)
    mom6_rates[i], process_mom_jacobian[i,2,:] = rate_out
    rate_out = moment_rate_and_grad(9, y, dfdt_val, dls, kernel, sum_idxs, num_sum_bins,
                                    bin_midpoints, 1.e-6, 1.e-3, m3_init, tscale)
    mom9_rates[i], process_mom_jacobian[i,3,:] = rate_out

In [ ]:
fig = plt.figure(figsize=(5, 4))
plt.plot(t_eval * tscale, mom9_rates)

In [ ]:
sigma_mom0_rate = np.zeros((num_time_steps+1,))
sigma_mom3_rate = np.zeros((num_time_steps+1,))
sigma_mom6_rate = np.zeros((num_time_steps+1,))
sigma_mom9_rate = np.zeros((num_time_steps+1,))
for i in range(num_time_steps+1):
    mom_inv = la.pinv(mom_jacobian[i,:,:])
    pj = process_mom_jacobian[i,:,:]
    pj[0,:] = mom0_rates[i] / mom0s[i] * (pj[0,:] - mom_jacobian[i,0,:])
    pj[1,:] = mom3_rates[i] / mom3s[i] * (pj[1,:] - mom_jacobian[i,1,:])
    pj[2,:] = mom6_rates[i] / mom6s[i] * (pj[2,:] - mom_jacobian[i,2,:])
    pj[3,:] = mom9_rates[i] / mom9s[i] * (pj[3,:] - mom_jacobian[i,3,:])
    cov = pj @ mom_inv @ np.eye(4) @ mom_inv.T @ pj.T
    sigma_mom0_rate[i] = np.sqrt(cov[0,0])
    sigma_mom3_rate[i] = np.sqrt(cov[1,1])
    sigma_mom6_rate[i] = np.sqrt(cov[2,2])
    sigma_mom9_rate[i] = np.sqrt(cov[3,3])

In [ ]:
fig = plt.figure(figsize=(5, 4))
plt.plot(t_eval * tscale, sigma_mom0_rate, label="M0")
#plt.plot(t_eval * tscale, sigma_mom3_rate)
plt.plot(t_eval * tscale, sigma_mom6_rate, label="M6")
plt.plot(t_eval * tscale, sigma_mom9_rate, label="M9")
ymax = 0.1
for i in range(num_time_steps+1):
    if autos[i] < accrs[i]:
        time = t_eval[i] * tscale
        break
plt.vlines(time, 0., ymax, 'k', label='Accr > Auto')
for i in range(num_time_steps+1):
    if autos[i] > 0.01 * autos.max():
        time = t_eval[i] * tscale
        break
plt.vlines(time, 0., ymax, 'r', linestyle='--', label='Auto at 1% max')
plt.ylim([0., ymax])
plt.legend(loc='best')
plt.savefig('m0369_predictability.png')

In [ ]:
mom4_rates = np.zeros((num_time_steps+1,))
mom5_rates = np.zeros((num_time_steps+1,))
process_small_jacobian = np.zeros((num_time_steps+1, 4, 3))
process_small_jacobian[:,:2,:] = process_mom_jacobian[:,:2,:]
for i in range(num_time_steps+1):
    y = ivp_out.y[:,i]
    dfdt_val = dfdt(0., y[:num_bins], dls, kernel, sum_idxs, num_sum_bins)
    rate_out = moment_rate_and_grad(4, y, dfdt_val, dls, kernel, sum_idxs, num_sum_bins,
                                    bin_midpoints, 1.e-6, 1.e-3, m3_init, tscale)
    mom4_rates[i], process_small_jacobian[i,2,:] = rate_out
    rate_out = moment_rate_and_grad(5, y, dfdt_val, dls, kernel, sum_idxs, num_sum_bins,
                                    bin_midpoints, 1.e-6, 1.e-3, m3_init, tscale)
    mom5_rates[i], process_small_jacobian[i,3,:] = rate_out

In [ ]:
sigma_small_mom0_rate = np.zeros((num_time_steps+1,))
sigma_small_mom3_rate = np.zeros((num_time_steps+1,))
sigma_small_mom4_rate = np.zeros((num_time_steps+1,))
sigma_small_mom5_rate = np.zeros((num_time_steps+1,))
for i in range(num_time_steps+1):
    mom_inv = la.pinv(mom_small_jacobian[i,:,:])
    pj = process_small_jacobian[i,:,:]
    pj[0,:] = mom0_rates[i] / mom0s[i] * (pj[0,:] - mom_small_jacobian[i,0,:])
    pj[1,:] = mom3_rates[i] / mom3s[i] * (pj[1,:] - mom_small_jacobian[i,1,:])
    pj[2,:] = mom4_rates[i] / mom4s[i] * (pj[2,:] - mom_small_jacobian[i,2,:])
    pj[3,:] = mom5_rates[i] / mom5s[i] * (pj[3,:] - mom_small_jacobian[i,3,:])
    cov = pj @ mom_inv @ np.eye(4) @ mom_inv.T @ pj.T
    sigma_small_mom0_rate[i] = np.sqrt(cov[0,0])
    sigma_small_mom3_rate[i] = np.sqrt(cov[1,1])
    sigma_small_mom4_rate[i] = np.sqrt(cov[2,2])
    sigma_small_mom5_rate[i] = np.sqrt(cov[3,3])

In [ ]:
fig = plt.figure(figsize=(5, 4))
plt.plot(t_eval * tscale, sigma_small_mom0_rate, label="M0")
#plt.plot(t_eval * tscale, sigma_small_mom3_rate)
plt.plot(t_eval * tscale, sigma_small_mom4_rate, label="M4")
plt.plot(t_eval * tscale, sigma_small_mom5_rate, label="M5")
ymax = 0.1
for i in range(num_time_steps+1):
    if autos[i] < accrs[i]:
        time = t_eval[i] * tscale
        break
plt.vlines(time, 0., ymax, 'k', label='Accr > Auto')
for i in range(num_time_steps+1):
    if autos[i] > 0.01 * autos.max():
        time = t_eval[i] * tscale
        break
plt.vlines(time, 0., ymax, 'r', linestyle='--', label='Auto at 1% max')
plt.ylim([0., ymax])
plt.legend(loc='best')
plt.savefig('m0345_predictability.png')

In [ ]:
fig = plt.figure(figsize=(5, 4))
ax = fig.gca()
ax.plot(t_eval * tscale, 10.*np.log10(mom5s), 'k')
ax.plot(t_eval * tscale, 10.*np.log10(mom5_rates), 'r--')

In [ ]:
fig = plt.figure(figsize=(5, 4))
ax = fig.gca()
ax.plot(t_eval * tscale, 10.*np.log10(mom9s), 'k')
ax.plot(t_eval * tscale, 10.*np.log10(mom9_rates), 'r--')

In [ ]:
momc0_rates = np.zeros((num_time_steps+1,))
momc3_rates = np.zeros((num_time_steps+1,))
momr0_rates = np.zeros((num_time_steps+1,))
momr3_rates = np.zeros((num_time_steps+1,))
process_2cat_jacobian = np.zeros((num_time_steps+1, 4, 3))
for i in range(num_time_steps+1):
    y = ivp_out.y[:,i]
    dfdt_val = dfdt(0., y[:num_bins], dls, kernel, sum_idxs, num_sum_bins)
    rate_out = moment_rate_and_grad(0, y, dfdt_val, dls, kernel, sum_idxs, num_sum_bins,
                                    bin_midpoints, 1.e-6, 1.e-4, m3_init, tscale)
    momc0_rates[i], process_2cat_jacobian[i,0,:] = rate_out
    rate_out = moment_rate_and_grad(3, y, dfdt_val, dls, kernel, sum_idxs, num_sum_bins,
                                    bin_midpoints, 1.e-6, 1.e-4, m3_init, tscale)
    momc3_rates[i], process_2cat_jacobian[i,1,:] = rate_out
    rate_out = moment_rate_and_grad(0, y, dfdt_val, dls, kernel, sum_idxs, num_sum_bins,
                                    bin_midpoints, 1.e-4, 1.e-3, m3_init, tscale)
    momr0_rates[i], process_2cat_jacobian[i,2,:] = rate_out
    rate_out = moment_rate_and_grad(3, y, dfdt_val, dls, kernel, sum_idxs, num_sum_bins,
                                    bin_midpoints, 1.e-4, 1.e-3, m3_init, tscale)
    momr3_rates[i], process_2cat_jacobian[i,3,:] = rate_out

In [ ]:
(momr3s[46]-momr3s[45])/(dt)

In [ ]:
print(rps[45])

In [ ]:
print(mom_2cat_jacobian[45,3,0]*momr3s[45]/tscale)

In [ ]:
print(momr3_rates[45])

In [ ]:
sigma_2cat_momc0_rate = np.zeros((num_time_steps+1,))
sigma_2cat_momc3_rate = np.zeros((num_time_steps+1,))
sigma_2cat_momr0_rate = np.zeros((num_time_steps+1,))
sigma_2cat_momr3_rate = np.zeros((num_time_steps+1,))
for i in range(num_time_steps+1):
    mom_inv = la.pinv(mom_2cat_jacobian[i,:,:])
    pj = process_2cat_jacobian[i,:,:]
    pj[0,:] = momc0_rates[i] / momc0s[i] * (pj[0,:] - mom_2cat_jacobian[i,0,:])
    pj[1,:] = momc3_rates[i] / momc3s[i] * (pj[1,:] - mom_2cat_jacobian[i,1,:])
    pj[2,:] = momr0_rates[i] / momr0s[i] * (pj[2,:] - mom_2cat_jacobian[i,2,:])
    pj[3,:] = momr3_rates[i] / momr3s[i] * (pj[3,:] - mom_2cat_jacobian[i,3,:])
    cov = pj @ mom_inv @ np.eye(4) @ mom_inv.T @ pj.T
    sigma_2cat_momc0_rate[i] = np.sqrt(cov[0,0])
    sigma_2cat_momc3_rate[i] = np.sqrt(cov[1,1])
    sigma_2cat_momr0_rate[i] = np.sqrt(cov[2,2])
    sigma_2cat_momr3_rate[i] = np.sqrt(cov[3,3])

In [ ]:
fig = plt.figure(figsize=(5, 4))
plt.plot(t_eval * tscale, sigma_2cat_momc0_rate, label="Cloud M0")
#plt.plot(t_eval * tscale, sigma_2cat_momc3_rate)
plt.plot(t_eval * tscale, sigma_2cat_momr0_rate, label="Rain M0")
plt.plot(t_eval * tscale, sigma_2cat_momr3_rate, label="Rain M3")
ymax = 0.1
for i in range(num_time_steps+1):
    if autos[i] < accrs[i]:
        time = t_eval[i] * tscale
        break
plt.vlines(time, 0., ymax, 'k', label='Accr > Auto')
for i in range(num_time_steps+1):
    if autos[i] > 0.01 * autos.max():
        time = t_eval[i] * tscale
        break
plt.vlines(time, 0., ymax, 'r', linestyle='--', label='Auto at 1% max')
plt.ylim([0., ymax])
plt.legend(loc='best')
plt.savefig('cm03_rm03_predictability.png')

In [ ]:
process_6_jacobian = np.zeros((num_time_steps+1, 6, 3))
process_6_jacobian[:,0,:] = process_small_jacobian[:,0,:]
process_6_jacobian[:,3:,:] = process_small_jacobian[:,1:,:]
mom1_rates = np.zeros((num_time_steps+1,))
mom2_rates = np.zeros((num_time_steps+1,))
for i in range(num_time_steps+1):
    y = ivp_out.y[:,i]
    dfdt_val = dfdt(0., y[:num_bins], dls, kernel, sum_idxs, num_sum_bins)
    rate_out = moment_rate_and_grad(1, y, dfdt_val, dls, kernel, sum_idxs, num_sum_bins,
                                    bin_midpoints, 1.e-6, 1.e-3, m3_init, tscale)
    mom1_rates[i], process_small_jacobian[i,1,:] = rate_out
    rate_out = moment_rate_and_grad(2, y, dfdt_val, dls, kernel, sum_idxs, num_sum_bins,
                                    bin_midpoints, 1.e-6, 1.e-3, m3_init, tscale)
    mom2_rates[i], process_small_jacobian[i,2,:] = rate_out

In [ ]:
sigma_6_mom0_rate = np.zeros((num_time_steps+1,))
sigma_6_mom1_rate = np.zeros((num_time_steps+1,))
sigma_6_mom2_rate = np.zeros((num_time_steps+1,))
sigma_6_mom3_rate = np.zeros((num_time_steps+1,))
sigma_6_mom4_rate = np.zeros((num_time_steps+1,))
sigma_6_mom5_rate = np.zeros((num_time_steps+1,))
for i in range(num_time_steps+1):
    mom_inv = la.pinv(mom_6_jacobian[i,:,:])
    pj = process_6_jacobian[i,:,:]
    pj[0,:] = mom0_rates[i] / mom0s[i] * (pj[0,:] - mom_6_jacobian[i,0,:])
    pj[1,:] = mom1_rates[i] / mom1s[i] * (pj[1,:] - mom_6_jacobian[i,1,:])
    pj[2,:] = mom2_rates[i] / mom2s[i] * (pj[2,:] - mom_6_jacobian[i,2,:])
    pj[3,:] = mom3_rates[i] / mom3s[i] * (pj[3,:] - mom_6_jacobian[i,3,:])
    pj[4,:] = mom4_rates[i] / mom4s[i] * (pj[4,:] - mom_6_jacobian[i,4,:])
    pj[5,:] = mom5_rates[i] / mom5s[i] * (pj[5,:] - mom_6_jacobian[i,5,:])
    cov = pj @ mom_inv @ np.eye(6) @ mom_inv.T @ pj.T
    sigma_6_mom0_rate[i] = np.sqrt(cov[0,0])
    sigma_6_mom1_rate[i] = np.sqrt(cov[1,1])
    sigma_6_mom2_rate[i] = np.sqrt(cov[2,2])
    sigma_6_mom3_rate[i] = np.sqrt(cov[3,3])
    sigma_6_mom4_rate[i] = np.sqrt(cov[4,4])
    sigma_6_mom5_rate[i] = np.sqrt(cov[5,5])

In [ ]:
fig = plt.figure(figsize=(5, 4))
plt.plot(t_eval * tscale, sigma_6_mom0_rate, label="M0")
plt.plot(t_eval * tscale, sigma_6_mom1_rate, label="M1")
plt.plot(t_eval * tscale, sigma_6_mom2_rate, label="M2")
plt.plot(t_eval * tscale, sigma_6_mom4_rate, label="M4")
plt.plot(t_eval * tscale, sigma_6_mom5_rate, label="M5")
ymax = 0.1
for i in range(num_time_steps+1):
    if autos[i] < accrs[i]:
        time = t_eval[i] * tscale
        break
plt.vlines(time, 0., ymax, 'k', label='Accr > Auto')
for i in range(num_time_steps+1):
    if autos[i] > 0.01 * autos.max():
        time = t_eval[i] * tscale
        break
plt.vlines(time, 0., ymax, 'r', linestyle='--', label='Auto at 1% max')
plt.ylim([0., ymax])
plt.legend(loc='best')
plt.savefig('m012345_predictability.png')

In [ ]:
momc6s = np.zeros((num_time_steps+1,))
momr6s = np.zeros((num_time_steps+1,))
mom_2cat_3m_jacobian = np.zeros((num_time_steps+1, 6, 3))
mom_2cat_3m_jacobian[:,:2,:] = mom_2cat_jacobian[:,:2,:]
mom_2cat_3m_jacobian[:,3:5,:] = mom_2cat_jacobian[:,2:,:]
for i in range(num_time_steps+1):
    y = ivp_out.y[:,i]
    dfdt_val = dfdt(0., y[:num_bins], dls, kernel, sum_idxs, num_sum_bins)
    mom_out = calc_moment_and_gradient(y, dfdt_val, 6, m3_init, bin_bounds, dls,
                                       bin_midpoints, 1.e-6, 1.e-4)
    momc6s[i], mom_2cat_3m_jacobian[i,2,:] = mom_out
    mom_out = calc_moment_and_gradient(y, dfdt_val, 6, m3_init, bin_bounds, dls,
                                       bin_midpoints, 1.e-4, 1.e-3)
    momr6s[i], mom_2cat_3m_jacobian[i,5,:] = mom_out

In [ ]:
momc6_rates = np.zeros((num_time_steps+1,))
momr6_rates = np.zeros((num_time_steps+1,))
process_2cat_3m_jacobian = np.zeros((num_time_steps+1, 6, 3))
process_2cat_3m_jacobian[:,:2,:] = process_2cat_jacobian[:,:2,:]
process_2cat_3m_jacobian[:,3:5,:] = process_2cat_jacobian[:,2:,:]
for i in range(num_time_steps+1):
    y = ivp_out.y[:,i]
    dfdt_val = dfdt(0., y[:num_bins], dls, kernel, sum_idxs, num_sum_bins)
    rate_out = moment_rate_and_grad(6, y, dfdt_val, dls, kernel, sum_idxs, num_sum_bins,
                                    bin_midpoints, 1.e-6, 1.e-4, m3_init, tscale)
    momc6_rates[i], process_2cat_3m_jacobian[i,2,:] = rate_out
    rate_out = moment_rate_and_grad(6, y, dfdt_val, dls, kernel, sum_idxs, num_sum_bins,
                                    bin_midpoints, 1.e-4, 1.e-3, m3_init, tscale)
    momr6_rates[i], process_2cat_3m_jacobian[i,5,:] = rate_out

In [ ]:
sigma_2cat_3m_momc0_rate = np.zeros((num_time_steps+1,))
sigma_2cat_3m_momc3_rate = np.zeros((num_time_steps+1,))
sigma_2cat_3m_momc6_rate = np.zeros((num_time_steps+1,))
sigma_2cat_3m_momr0_rate = np.zeros((num_time_steps+1,))
sigma_2cat_3m_momr3_rate = np.zeros((num_time_steps+1,))
sigma_2cat_3m_momr6_rate = np.zeros((num_time_steps+1,))
for i in range(num_time_steps+1):
    mom_inv = la.pinv(mom_2cat_3m_jacobian[i,:,:])
    pj = process_2cat_3m_jacobian[i,:,:]
    pj[0,:] = momc0_rates[i] / momc0s[i] * (pj[0,:] - mom_2cat_3m_jacobian[i,0,:])
    pj[1,:] = momc3_rates[i] / momc3s[i] * (pj[1,:] - mom_2cat_3m_jacobian[i,1,:])
    pj[2,:] = momc6_rates[i] / momc6s[i] * (pj[2,:] - mom_2cat_3m_jacobian[i,2,:])
    pj[3,:] = momr0_rates[i] / momr0s[i] * (pj[3,:] - mom_2cat_3m_jacobian[i,3,:])
    pj[4,:] = momr3_rates[i] / momr3s[i] * (pj[4,:] - mom_2cat_3m_jacobian[i,4,:])
    pj[5,:] = momr6_rates[i] / momr6s[i] * (pj[5,:] - mom_2cat_3m_jacobian[i,5,:])
    cov = pj @ mom_inv @ np.eye(6) @ mom_inv.T @ pj.T
    sigma_2cat_3m_momc0_rate[i] = np.sqrt(cov[0,0])
    sigma_2cat_3m_momc3_rate[i] = np.sqrt(cov[1,1])
    sigma_2cat_3m_momc6_rate[i] = np.sqrt(cov[2,2])
    sigma_2cat_3m_momr0_rate[i] = np.sqrt(cov[3,3])
    sigma_2cat_3m_momr3_rate[i] = np.sqrt(cov[4,4])
    sigma_2cat_3m_momr6_rate[i] = np.sqrt(cov[5,5])

In [ ]:
fig = plt.figure(figsize=(5, 4))
plt.plot(t_eval * tscale, sigma_2cat_3m_momc0_rate, label="Cloud M0")
#plt.plot(t_eval * tscale, sigma_2cat_momc3_rate)
plt.plot(t_eval * tscale, sigma_2cat_3m_momc6_rate, label="Cloud M6")
plt.plot(t_eval * tscale, sigma_2cat_3m_momr0_rate, label="Rain M0")
plt.plot(t_eval * tscale, sigma_2cat_3m_momr3_rate, label="Rain M3")
plt.plot(t_eval * tscale, sigma_2cat_3m_momr6_rate, label="Rain M6")
ymax = 0.1
for i in range(num_time_steps+1):
    if autos[i] < accrs[i]:
        time = t_eval[i] * tscale
        break
plt.vlines(time, 0., ymax, 'k', label='Accr > Auto')
for i in range(num_time_steps+1):
    if autos[i] > 0.01 * autos.max():
        time = t_eval[i] * tscale
        break
plt.vlines(time, 0., ymax, 'r', linestyle='--', label='Auto at 1% max')
plt.ylim([0., ymax])
plt.legend(loc='best')
plt.savefig('cm036_rm036_predictability.png')

# Comparison plots

In [ ]:
for i in range(num_time_steps+1):
    if momr3s[i] >= 0.99*mom3s[i]:
        half_time = i//2
        break

In [ ]:
t_eval[half_time]*tscale

In [ ]:
nmom = 10
full_moms = np.zeros((num_time_steps+1, nmom))
full_mom_rates = np.zeros((num_time_steps+1, nmom))
full_mom_jacobian = np.zeros((num_time_steps+1, nmom, 3))
full_mom_rate_jacobian = np.zeros((num_time_steps+1, nmom, 3))
for i in range(num_time_steps+1):
    y = ivp_out.y[:,i]
    dfdt_val = dfdt(0., y[:num_bins], dls, kernel, sum_idxs, num_sum_bins)
    for n in range(nmom):
        mom_out = calc_moment_and_gradient(y, dfdt_val, n, m3_init, bin_bounds, dls,
                                           bin_midpoints, 1.e-6, 1.e-3)
        full_moms[i,n], full_mom_jacobian[i,n,:] = mom_out
        rate_out = moment_rate_and_grad(n, y, dfdt_val, dls, kernel, sum_idxs, num_sum_bins,
                                        bin_midpoints, 1.e-6, 1.e-3, m3_init, tscale)
        full_mom_rates[i,n], full_mom_rate_jacobian[i,n,:] = rate_out

In [ ]:
full_scaled_rate_jacobian = np.zeros((num_time_steps+1, nmom, 3))
for i in range(num_time_steps+1):
    mj = full_mom_jacobian[i,:,:]
    pj = full_mom_rate_jacobian[i,:,:]
    for n in range(nmom):
        full_scaled_rate_jacobian[i,n,:] = full_mom_rates[i,n] / full_moms[i,n] * (pj[n,:] - mj[n,:])

In [ ]:
moments = [0, 4, 5]
nmom = len(moments)
j_eigs = np.zeros((num_time_steps+1, nmom), dtype=np.complex128)
j_evals = np.zeros((num_time_steps+1, nmom, nmom), dtype=np.complex128)
for i in range(num_time_steps):
    mom_inv = la.pinv(full_mom_jacobian[i,moments,:])
    pj = full_scaled_rate_jacobian[i,moments,:]
    j_eigs[i,:], j_evals[i,:,:] = la.eig(pj @ mom_inv)

In [ ]:
print(j_eigs[0,:], j_evals[0,:,:])

In [ ]:
plt.scatter(np.repeat(t_eval * tscale, len(moments)), np.ravel(np.real(j_eigs)))

In [ ]:
plt.scatter(np.repeat(t_eval * tscale, len(moments)), np.ravel(np.imag(j_eigs)))

In [ ]:
full_mom_jacobian[0,[0,3,6,9],:]

In [ ]:
mom_jacobian[0,:,:]

In [ ]:
moments = [0, 6, 9]
nmom = len(moments)
sigma_diags = np.zeros((num_time_steps+1, nmom))
for i in range(num_time_steps):
    mom_inv = la.pinv(full_mom_jacobian[i,moments,:])
    pj = full_scaled_rate_jacobian[i,moments,:]
    cov = pj @ mom_inv @ np.eye(nmom) @ mom_inv.T @ pj.T
    for n in range(nmom):
        sigma_diags[i,n] = np.sqrt(cov[n,n])

In [ ]:
fig = plt.figure(figsize=(5, 4))
for n in range(nmom):
    plt.plot(t_eval * tscale, sigma_diags[:,n], label="M{}".format(moments[n]))
ymax = 0.1
for i in range(num_time_steps+1):
    if autos[i] < accrs[i]:
        time = t_eval[i] * tscale
        break
plt.vlines(time, 0., ymax, 'k', label='Accr > Auto')
for i in range(num_time_steps+1):
    if autos[i] > 0.01 * autos.max():
        time = t_eval[i] * tscale
        break
plt.vlines(time, 0., ymax, 'r', linestyle='--', label='Auto at 1% max')
plt.ylim([0., ymax])
plt.legend(loc='best')

In [ ]:
initial_error = 0.0005 * np.log(10.)
error_per_sec = 0.005 * np.log(10.) / 60.

In [ ]:
moments = [0, 6, 9]
nmom = len(moments)
sigma_diags = np.zeros((num_time_steps+1, nmom))
cum_cov = initial_error * np.eye(nmom)
for i in range(num_time_steps):
    cum_cov += dt * error_per_sec * np.eye(nmom)
    mom_inv = la.pinv(full_mom_jacobian[i,moments,:])
    pj = full_scaled_rate_jacobian[i,moments,:]
    h = (np.eye(nmom) + dt * (pj @ mom_inv))
    new_cov = h @ cum_cov @ h.T
    cum_cov = new_cov
    for n in range(nmom):
        sigma_diags[i,n] = np.sqrt(cum_cov[n,n])

In [ ]:
fig = plt.figure(figsize=(5, 4))
for n in range(nmom):
    plt.plot(t_eval * tscale, sigma_diags[:,n], label="M{}".format(moments[n]))
ymax = 20.
for i in range(num_time_steps+1):
    if autos[i] < accrs[i]:
        time = t_eval[i] * tscale
        break
plt.vlines(time, 0., ymax, 'k', label='Accr > Auto')
for i in range(num_time_steps+1):
    if autos[i] > 0.01 * autos.max():
        time = t_eval[i] * tscale
        break
plt.vlines(time, 0., ymax, 'r', linestyle='--', label='Auto at 1% max')
plt.ylim([0., ymax])
plt.legend(loc='best')

In [ ]:
moments = [0, 1, 2]
nmom = len(moments)
sigma_diags = np.zeros((num_time_steps+1, nmom))
cum_cov = initial_error * np.eye(nmom)
for i in range(num_time_steps):
    cum_cov += dt * error_per_sec * np.eye(nmom)
    mom_inv = la.pinv(full_mom_jacobian[i,moments,:])
    pj = full_scaled_rate_jacobian[i,moments,:]
    h = (np.eye(nmom) + dt * (pj @ mom_inv))
    new_cov = h @ cum_cov @ h.T
    cum_cov = new_cov
    for n in range(nmom):
        sigma_diags[i,n] = np.sqrt(cum_cov[n,n])

In [ ]:
fig = plt.figure(figsize=(5, 4))
for n in range(nmom):
    plt.plot(t_eval * tscale, sigma_diags[:,n], label="M{}".format(moments[n]))
ymax = 20.
for i in range(num_time_steps+1):
    if autos[i] < accrs[i]:
        time = t_eval[i] * tscale
        break
plt.vlines(time, 0., ymax, 'k', label='Accr > Auto')
for i in range(num_time_steps+1):
    if autos[i] > 0.01 * autos.max():
        time = t_eval[i] * tscale
        break
plt.vlines(time, 0., ymax, 'r', linestyle='--', label='Auto at 1% max')
plt.ylim([0., ymax])
plt.legend(loc='best')

In [ ]:
moments = [0, 4, 5]
nmom = len(moments)
sigma_diags = np.zeros((num_time_steps+1, nmom))
cum_cov = initial_error * np.eye(nmom)
for i in range(num_time_steps):
    cum_cov += dt * error_per_sec * np.eye(nmom)
    mom_inv = la.pinv(full_mom_jacobian[i,moments,:])
    pj = full_scaled_rate_jacobian[i,moments,:]
    h = (np.eye(nmom) + dt * (pj @ mom_inv))
    new_cov = h @ cum_cov @ h.T
    cum_cov = new_cov
    for n in range(nmom):
        sigma_diags[i,n] = np.sqrt(cum_cov[n,n])

In [ ]:
fig = plt.figure(figsize=(5, 4))
for n in range(nmom):
    plt.plot(t_eval * tscale, sigma_diags[:,n], label="M{}".format(moments[n]))
ymax = 20.
for i in range(num_time_steps+1):
    if autos[i] < accrs[i]:
        time = t_eval[i] * tscale
        break
plt.vlines(time, 0., ymax, 'k', label='Accr > Auto')
for i in range(num_time_steps+1):
    if autos[i] > 0.01 * autos.max():
        time = t_eval[i] * tscale
        break
plt.vlines(time, 0., ymax, 'r', linestyle='--', label='Auto at 1% max')
plt.ylim([0., ymax])
plt.legend(loc='best')

In [ ]:
moments = [0, 5, 6]
nmom = len(moments)
sigma_diags = np.zeros((num_time_steps+1, nmom))
cum_cov = initial_error * np.eye(nmom)
for i in range(num_time_steps):
    cum_cov += dt * error_per_sec * np.eye(nmom)
    mom_inv = la.pinv(full_mom_jacobian[i,moments,:])
    pj = full_scaled_rate_jacobian[i,moments,:]
    h = (np.eye(nmom) + dt * (pj @ mom_inv))
    new_cov = h @ cum_cov @ h.T
    cum_cov = new_cov
    for n in range(nmom):
        sigma_diags[i,n] = np.sqrt(cum_cov[n,n])

In [ ]:
fig = plt.figure(figsize=(5, 4))
for n in range(nmom):
    plt.plot(t_eval * tscale, sigma_diags[:,n], label="M{}".format(moments[n]))
ymax = 20.
for i in range(num_time_steps+1):
    if autos[i] < accrs[i]:
        time = t_eval[i] * tscale
        break
plt.vlines(time, 0., ymax, 'k', label='Accr > Auto')
for i in range(num_time_steps+1):
    if autos[i] > 0.01 * autos.max():
        time = t_eval[i] * tscale
        break
plt.vlines(time, 0., ymax, 'r', linestyle='--', label='Auto at 1% max')
plt.ylim([0., ymax])
plt.legend(loc='best')

# Accumulating random errors

In [ ]:
import pymc3 as pm
half_decibel = 0.05 * np.log(10)

In [ ]:
moments = [0, 6, 9]
nmom = len(moments)
errs = np.zeros((num_time_steps+1, nmom))
cum_err = np.zeros(nmom)
err_source = pm.MvNormal.dist(mu=np.zeros((nmom,)), cov=np.eye(nmom), shape=(nmom,))
for i in range(num_time_steps):
    cum_err += dt * half_decibel * err_source.random(1)
    mom_inv = la.pinv(full_mom_jacobian[i,moments,:])
    pj = full_scaled_rate_jacobian[i,moments,:]
    new_err = pj @ mom_inv @ cum_err
    cum_err += dt * new_err
    for n in range(nmom):
        errs[i,n] = np.abs(cum_err[n])

In [ ]:
fig = plt.figure(figsize=(5, 4))
for n in range(nmom):
    plt.plot(t_eval * tscale, errs[:,n], label="M{}".format(moments[n]))
ymax = 100
for i in range(num_time_steps+1):
    if autos[i] < accrs[i]:
        time = t_eval[i] * tscale
        break
plt.vlines(time, 0., ymax, 'k', label='Accr > Auto')
for i in range(num_time_steps+1):
    if autos[i] > 0.01 * autos.max():
        time = t_eval[i] * tscale
        break
plt.vlines(time, 0., ymax, 'r', linestyle='--', label='Auto at 1% max')
plt.ylim([0., ymax])
plt.legend(loc='best')

In [ ]:
moments = [0, 1, 2]
nmom = len(moments)
errs = np.zeros((num_time_steps+1, nmom))
cum_err = np.zeros(nmom)
err_source = pm.MvNormal.dist(mu=np.zeros((nmom,)), cov=np.eye(nmom), shape=(nmom,))
for i in range(num_time_steps):
    cum_err += dt * half_decibel * err_source.random(1)
    mom_inv = la.pinv(full_mom_jacobian[i,moments,:])
    pj = full_scaled_rate_jacobian[i,moments,:]
    new_err = pj @ mom_inv @ cum_err
    cum_err += dt * new_err
    for n in range(nmom):
        errs[i,n] = np.abs(cum_err[n])

In [ ]:
fig = plt.figure(figsize=(5, 4))
for n in range(nmom):
    plt.plot(t_eval * tscale, errs[:,n], label="M{}".format(moments[n]))
ymax = 100
for i in range(num_time_steps+1):
    if autos[i] < accrs[i]:
        time = t_eval[i] * tscale
        break
plt.vlines(time, 0., ymax, 'k', label='Accr > Auto')
for i in range(num_time_steps+1):
    if autos[i] > 0.01 * autos.max():
        time = t_eval[i] * tscale
        break
plt.vlines(time, 0., ymax, 'r', linestyle='--', label='Auto at 1% max')
plt.ylim([0., ymax])
plt.legend(loc='best')

In [ ]:
moments = [0, 4, 5]
nmom = len(moments)
errs = np.zeros((num_time_steps+1, nmom))
cum_err = np.zeros(nmom)
err_source = pm.MvNormal.dist(mu=np.zeros((nmom,)), cov=np.eye(nmom), shape=(nmom,))
for i in range(num_time_steps):
    cum_err += dt * half_decibel * err_source.random(1)
    mom_inv = la.pinv(full_mom_jacobian[i,moments,:])
    pj = full_scaled_rate_jacobian[i,moments,:]
    new_err = pj @ mom_inv @ cum_err
    cum_err += dt * new_err
    for n in range(nmom):
        errs[i,n] = np.abs(cum_err[n])

In [ ]:
fig = plt.figure(figsize=(5, 4))
for n in range(nmom):
    plt.plot(t_eval * tscale, errs[:,n], label="M{}".format(moments[n]))
ymax = 100
for i in range(num_time_steps+1):
    if autos[i] < accrs[i]:
        time = t_eval[i] * tscale
        break
plt.vlines(time, 0., ymax, 'k', label='Accr > Auto')
for i in range(num_time_steps+1):
    if autos[i] > 0.01 * autos.max():
        time = t_eval[i] * tscale
        break
plt.vlines(time, 0., ymax, 'r', linestyle='--', label='Auto at 1% max')
plt.ylim([0., ymax])
plt.legend(loc='best')